In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl

In [17]:
with open("results_1.pkl", "rb") as f:
    results = pkl.load(f)

In [18]:
results

{'train_losses': [4.501920859018962],
 'avg_train_accuracies': [0.011111111380159855],
 'test_losses': [4.499199787775676],
 'test_accuracies': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0.], dtype=float32),
 'avg_test_accuracies': [0.011111111380159855],
 'test_precision': array([0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.     

In [1]:
def plot_loss_accuracy(results) -> None:
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16, 4))
    fig.subplots_adjust(wspace=0.4)

    # ax[0].set_title("Loss")
    # ax[0].set_xlabel("Epoch")
    # ax[0].plot(results[0][0], label="Train Loss", color="blue")
    # ax[0].plot(results[1][0], label="Test Loss", color="red")
    # ax[0].legend()
    # ax[0].set_ylabel("Cross Entropy Loss")

    # ax[1].set_title("Accuracy")
    # ax[1].set_xlabel("Epoch")
    # ax[1].plot(results[0][1], label="Train Accuracy", color="blue")
    # ax[1].plot(results[1][1], label="Test Accuracy", color="red")
    # ax[1].legend()
    # ax[1].set_ylabel("Accuracy (%)")

    # ax[2].set_title("Precision and Recall")
    # ax[2].set_xlabel("Epoch")
    # ax[2].plot(results[2][0], label="Test Precision", color="cyan")
    # ax[2].plot(results[2][1], label="Test Recall", color="orange")
    # ax[2].legend()
    # ax[2].set_ylabel("Precision/Recall Score (0-1)")

In [2]:
def plot_improvement_by_class(results) -> None:
    pass

In [3]:
def plot_diff_barchart(results) -> None:
    pass